### Imports

In [2]:
import duckdb
from core.utils import duck_to_pandas, expand_column
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from google.cloud import storage
import re

### Events

In [3]:
database_path = '/home/marcelo-borges/Documentos/Projetos/SIAN/laboratory/telemetry-validation/db/telemetria.duckdb'
query = "SELECT * FROM events"
df_events = duck_to_pandas(database_path, query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### EDA

In [4]:
df_events.sample(10)

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,StartPosition,Value,EndDateTime,StartOdometerKilometres,StartDateTime,EventCategory,FuelUsedLitres,EndPosition,EndOdometerKilometres,LocationId
94584,71,1003,7038227315454193211,3289607304982580060,-106647426425259773,1498924535635898368,NaN,48.0,2024-04-08T20:55:55Z,NaN,2024-04-08T19:56:26Z,Summary,NaN,NaN,NaN,NaN
1141902,1,2,5928173277131215037,3326349222525094636,6918014154823921341,1547007653687537664,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-07-16T20:13:52Z,579888.3,2024-07-16T20:13:50Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",579888.40,NaN
557608,1,9,-3393530750645328945,3306608196268359455,-106647426425259773,1514402039761068032,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-05-24T15:28:43Z,617260.6,2024-05-24T15:28:34Z,Detail,0.006000,NaN,NaN,NaN
849900,1,3,7651958308154428287,3316681904515721394,1491699298243870720,1498924807959474176,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",0.0,2024-06-20T18:59:45Z,906106.5,2024-06-20T18:59:42Z,Detail,NaN,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",906106.60,NaN
273931,1,1,-7417774485302453264,3296302593214690555,-106647426425259773,1514480648201252864,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",NaN,2024-04-26T20:57:35Z,487355.2,2024-04-26T20:57:34Z,Detail,NaN,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",487355.20,NaN
803995,1,13,-3393530750645328945,3314853179420403778,1491699300338892800,1498924408951140352,NaN,0.0,2024-06-15T20:43:32Z,NaN,2024-06-15T20:43:19Z,Detail,0.014963,NaN,NaN,NaN
1100072,1,8,-3393530750645328945,3325112920690890330,-106647426425259773,1498924246870650880,NaN,0.0,2024-07-13T12:16:29Z,NaN,2024-07-13T12:16:21Z,Detail,0.009797,NaN,NaN,NaN
810474,1,1,7651958308154428287,3315448321702510963,1491699294175395840,1498924535635898368,"{'AgeOfReadingSeconds': 5, 'AltitudeMetres': N...",0.0,2024-06-17T11:12:47Z,18636.3,2024-06-17T11:12:46Z,Detail,NaN,"{'AgeOfReadingSeconds': 5, 'AltitudeMetres': N...",18636.31,NaN
738624,1,4,7651958308154428287,3312830324992626031,1491699282997575680,1498924246870650880,"{'AgeOfReadingSeconds': 3, 'AltitudeMetres': N...",0.0,2024-06-10T09:53:40Z,924658.9,2024-06-10T09:53:36Z,Detail,NaN,"{'AgeOfReadingSeconds': 3, 'AltitudeMetres': N...",924658.90,NaN
1187941,1,1,1443761028319829598,3326919467955285029,-106647426425259773,1547011194263121920,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",1700.0,2024-07-18T09:06:42Z,847807.9,2024-07-18T09:06:41Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",847807.90,NaN


In [5]:
df_events.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,LocationId
count,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.205826e+06,9.181830e+05,381046.000000,6.378600e+05,1.206700e+04
mean,2.005445e+00,2.073852e+02,1.551935e+18,3.308199e+18,4.865914e+17,1.507137e+18,1.007198e+03,6.964791e+05,0.019021,6.924665e+05,1.508070e+18
std,1.317910e+01,1.397186e+03,5.584598e+18,1.285241e+16,1.217497e+18,1.386682e+16,4.812906e+03,5.094844e+05,0.045152,4.136210e+05,2.815040e+14
min,0.000000e+00,0.000000e+00,-8.078464e+18,3.275198e+18,-9.195140e+18,1.498924e+18,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.507979e+18
25%,1.000000e+00,1.000000e+00,-3.393531e+18,3.297818e+18,-1.066474e+17,1.498925e+18,0.000000e+00,5.489248e+05,0.003741,5.532453e+05,1.507980e+18
50%,1.000000e+00,3.000000e+00,1.443761e+18,3.308728e+18,-1.066474e+17,1.498925e+18,0.000000e+00,6.855360e+05,0.008728,8.499894e+05,1.507981e+18
75%,1.000000e+00,1.500000e+01,7.651958e+18,3.319500e+18,1.491699e+18,1.514417e+18,3.200000e-01,9.167366e+05,0.022444,9.222814e+05,1.507983e+18
max,8.950000e+02,9.269800e+04,8.678978e+18,3.327150e+18,9.206705e+18,1.547012e+18,3.111738e+04,4.050610e+06,5.566263,4.050610e+06,1.508963e+18


In [6]:
df_events.dtypes

TotalOccurances              int64
TotalTimeSeconds             int64
EventTypeId                  int64
EventId                      int64
DriverId                     int64
AssetId                      int64
StartPosition               object
Value                      float64
EndDateTime                 object
StartOdometerKilometres    float64
StartDateTime               object
EventCategory               object
FuelUsedLitres             float64
EndPosition                 object
EndOdometerKilometres      float64
LocationId                 float64
dtype: object

In [7]:
df_events.isna().sum()

TotalOccurances                  0
TotalTimeSeconds                 0
EventTypeId                      0
EventId                          0
DriverId                         0
AssetId                          0
StartPosition               320773
Value                        33129
EndDateTime                  26912
StartOdometerKilometres     320772
StartDateTime                    0
EventCategory                    0
FuelUsedLitres              857909
EndPosition                 601091
EndOdometerKilometres       601095
LocationId                 1226888
dtype: int64

### Uso de memória

#### original

In [8]:
specific_date = '2024-07-18'
# filtered_df = df_positions[df_positions['Timestamp'].dt.date == pd.to_datetime(specific_date).date()]
df_original = df_events[df_events['StartDateTime'].str.contains(specific_date)]

In [9]:
import builtins
memory_usage_bytes = df_original.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 58951920 bytes
Memória total usada pelo DataFrame: 56.22 MB
Memória total usada pelo DataFrame: 0.05 GB


In [10]:
df_original.to_csv('df_original.csv', sep=';',decimal=',')

#### sugerido

In [11]:
df_sugerido = df_original.copy()

In [12]:
start_position_expanded = expand_column(df_sugerido, 'StartPosition')
end_position_expanded = expand_column(df_sugerido, 'EndPosition')

In [13]:
df_expanded = pd.concat([df_sugerido.drop(columns=['StartPosition', 'EndPosition']), start_position_expanded.add_prefix('StartPosition_'), end_position_expanded.add_prefix('EndPosition_')], axis=1)


In [14]:
df_expanded.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,...,EndPosition_Hdop,EndPosition_Heading,EndPosition_Latitude,EndPosition_Longitude,EndPosition_NumberOfSatellites,EndPosition_OdometerKilometres,EndPosition_Pdop,EndPosition_PositionId,EndPosition_SpeedKilometresPerHour,EndPosition_SpeedLimit
count,60597.000000,60597.000000,6.059700e+04,6.059700e+04,6.059700e+04,6.059700e+04,60597.000000,48376.000000,11575.000000,37080.000000,...,37081.000000,37081.000000,37081.000000,37081.000000,37081.000000,37075.000000,37081.0,3.708100e+04,37081.000000,23495.000000
mean,1.596317,184.207023,7.567854e+17,3.327009e+18,2.443609e+17,1.537603e+18,1445.067649,666404.310789,0.013540,672653.788072,...,1.022842,172.926485,-16.737004,-49.288460,11.987837,672744.503670,0.0,3.327009e+18,22.309876,53.759098
std,9.689379,990.335863,3.880342e+18,9.175486e+13,3.056904e+18,1.806712e+16,3746.143316,234014.461953,0.023141,240205.128034,...,0.737273,103.877560,0.050117,0.044617,0.212571,240094.263288,0.0,9.001194e+13,12.358762,10.899364
min,0.000000,0.000000,-8.078464e+18,3.326779e+18,-8.976664e+18,1.498924e+18,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,-16.976239,-49.454449,5.000000,0.053000,0.0,3.326779e+18,0.000000,30.000000
25%,1.000000,1.000000,-3.393531e+18,3.326938e+18,-1.066474e+17,1.547008e+18,0.000000,580294.675000,0.003500,580331.100000,...,1.000000,79.000000,-16.766032,-49.318716,12.000000,580331.600000,0.0,3.326939e+18,13.663000,40.000000
50%,1.000000,2.000000,1.443761e+18,3.327016e+18,-1.066474e+17,1.547009e+18,800.000000,708414.900000,0.006500,733710.900000,...,1.000000,179.000000,-16.728280,-49.283877,12.000000,733711.100000,0.0,3.327015e+18,23.254000,60.000000
75%,1.000000,13.000000,1.443761e+18,3.327092e+18,3.118870e+17,1.547011e+18,1800.000000,835459.500000,0.016000,835553.425000,...,1.000000,253.000000,-16.702322,-49.258800,12.000000,835553.438000,0.0,3.327090e+18,31.514000,60.000000
max,527.000000,33309.000000,8.678978e+18,3.327150e+18,7.405822e+18,1.547012e+18,29313.470588,982848.200000,0.579000,982848.300000,...,25.000000,359.000000,-16.650273,-49.127992,12.000000,982848.300000,0.0,3.327150e+18,60.727000,110.000000


In [15]:
null_count = df_expanded.isnull().sum()
total_registers = df_expanded.shape[0]
percent_null = (null_count / total_registers) * 100

In [16]:
formatted_percent_null = [f"{percent:.2f}%" for percent in percent_null]

In [17]:
percent_null_df = pd.DataFrame({
    'Coluna': df_expanded.columns,
    'Percentual de Nulos': formatted_percent_null
})

In [18]:
percent_null_df.sort_values('Percentual de Nulos', ascending=False)

,Coluna,Percentual de Nulos
13,LocationId,99.38%
33,EndPosition_AltitudeMetres,99.38%
15,StartPosition_AltitudeMetres,99.38%
11,FuelUsedLitres,80.90%
48,EndPosition_SpeedLimit,61.23%
30,StartPosition_SpeedLimit,51.57%
43,EndPosition_OdometerKilometres,38.82%
34,EndPosition_AssetId,38.81%
35,EndPosition_DriverId,38.81%
36,EndPosition_FormattedAddress,38.81%


#### Sugestão de retirada de coluna

In [19]:
df_expanded.drop(columns=['EndPosition_AltitudeMetres','StartPosition_AltitudeMetres'], inplace=True)

In [20]:
df_expanded['EndDateTime'] = pd.to_datetime(df_expanded['EndDateTime'], errors='coerce')
df_expanded['StartDateTime'] = pd.to_datetime(df_expanded['StartDateTime'], errors='coerce')


In [21]:
float_to_int_columns = [
    'StartPosition_AgeOfReadingSeconds',
    'StartPosition_AssetId',
    'StartPosition_DriverId',
    'StartPosition_NumberOfSatellites',
    'StartPosition_PositionId',
    'EndPosition_AgeOfReadingSeconds',
    'EndPosition_AssetId',
    'EndPosition_DriverId',
    'EndPosition_NumberOfSatellites',
    'EndPosition_PositionId'
]

In [22]:
for column in float_to_int_columns:
    df_expanded[column] = df_expanded[column].astype('Int64', errors='ignore')


In [23]:
data_types_after_casting = df_expanded.dtypes
data_types_after_casting

TotalOccurances                                       int64
TotalTimeSeconds                                      int64
EventTypeId                                           int64
EventId                                               int64
DriverId                                              int64
AssetId                                               int64
Value                                               float64
EndDateTime                             datetime64[ns, UTC]
StartOdometerKilometres                             float64
StartDateTime                           datetime64[ns, UTC]
EventCategory                                        object
FuelUsedLitres                                      float64
EndOdometerKilometres                               float64
LocationId                                          float64
StartPosition_AgeOfReadingSeconds                     Int64
StartPosition_AssetId                                 Int64
StartPosition_DriverId                  

In [24]:
import builtins
memory_usage_bytes = df_expanded.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 55183892 bytes
Memória total usada pelo DataFrame: 52.63 MB
Memória total usada pelo DataFrame: 0.05 GB


In [44]:
df_expanded.to_csv('df_expanded.csv', sep=';',decimal=',')

In [45]:
df_expanded.columns

Index(['TotalOccurances', 'TotalTimeSeconds', 'EventTypeId', 'EventId',
       'DriverId', 'AssetId', 'Value', 'EndDateTime',
       'StartOdometerKilometres', 'StartDateTime', 'EventCategory',
       'FuelUsedLitres', 'EndOdometerKilometres', 'LocationId',
       'StartPosition_AgeOfReadingSeconds', 'StartPosition_AltitudeMetres',
       'StartPosition_AssetId', 'StartPosition_DriverId',
       'StartPosition_FormattedAddress', 'StartPosition_Hdop',
       'StartPosition_Heading', 'StartPosition_IsAvl',
       'StartPosition_Latitude', 'StartPosition_Longitude',
       'StartPosition_NumberOfSatellites', 'StartPosition_OdometerKilometres',
       'StartPosition_Pdop', 'StartPosition_PositionId',
       'StartPosition_Source', 'StartPosition_SpeedKilometresPerHour',
       'StartPosition_SpeedLimit', 'StartPosition_Timestamp',
       'EndPosition_AgeOfReadingSeconds', 'EndPosition_AltitudeMetres',
       'EndPosition_AssetId', 'EndPosition_DriverId',
       'EndPosition_FormattedAddres

In [25]:
df_original.to_csv('events_original.csv', sep=';',decimal=',')
df_expanded.to_csv('events_expanded.csv', sep=';',decimal=',')